# Implementacija predikcije ocene parfema uz koriscenje _ANN_

Pre svega je neophodno importovati sve potrebne biblioteke koje će nam služiti za analizu.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
import re
from ast import literal_eval
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-09-14 21:35:21.709846: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-14 21:35:21.710182: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-14 21:35:21.764018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-14 21:35:23.177963: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Funckija `parse_season_ratings()` sluzi za za parsiranje polja godisnja doba iz skupa podataka, dok funckija `consolidate_notes()` objedinjuje/konsoliduje base, middle i top note u jednu kolekciju.

Funkcija `parse_season_ratings()` služi za automatsko izdvajanje i konverziju sezonskih ocena iz tekstualnog stringa u strukturirani _Python_ rečnik. Korišćenjem regularnog izraza, iz stringa poput "_Summer: 85.0% Winter: 10.0%_" prepoznaje parove (sezona, procenat) i za svaku sezonu (npr. "_Summer_", "_Winter_") dodeljuje odgovarajući procenat kao numeričku vrednost. Rezultat je rečnik gde su ključevi nazivi sezona, a vrednosti procenti, što omogućava jednostavno filtriranje i analizu parfema po sezonskim preferencama.

Funkcija `consolidate_notes(notes)` objedinjuje sve mirisne note parfema u jednu listu, bez obzira na njihovu hijerarhiju (_top_, _middle_, _base_). Za svaki tip note proverava da li postoji u rečniku `notes` i, ukoliko postoji, dodaje sve note tog tipa u zajedničku listu. Na taj način se mirisni profil parfema pojednostavljuje i omogućava obrada i enkodiranje podataka za potrebe modela, bez gubitka informacija o prisutnim notama.

In [2]:
def parse_season_ratings(rating_str):
    pattern = r'([A-Za-z]+):\s*([0-9.]+)%'
    return {season: float(percent) for season, percent in re.findall(pattern, rating_str)}

def consolidate_notes(notes):
    all_notes = []
    for note_type in ['Top Notes', 'Middle Notes', 'Base Notes']:
        if note_type in notes:
            all_notes.extend(notes[note_type])
    return all_notes


Funkcije za izdvajanje _top_, _middle_ i _base_ nota iz polja _Notes_:

In [3]:
def extract_top_notes(x):
    if isinstance(x, dict):
        return x.get('Top Notes', [])
    elif isinstance(x, list):
        return []
    else:
        return []

def extract_middle_notes(x):
    if isinstance(x, dict):
        return x.get('Middle Notes', [])
    elif isinstance(x, list):
        return []
    else:
        return []

def extract_base_notes(x):
    if isinstance(x, dict):
        return x.get('Base Notes', [])
    elif isinstance(x, list):
        return x  # ako su sve note lista, tretiraj kao base
    else:
        return []

Učitavanje skupa podataka:

In [4]:
file_path = "../datasets/mainDataset.csv"
data = pd.read_csv(file_path, delimiter='|')

Izdvajanje relevantnih polja iz skupa podataka koji ce biti ulaz u algoritme. Prvobitno nismo dodali akorde, ali nakon dodavanja (da bi se uskladili sa ulaznim podacima u prethodne 2 metoda) nismo primetili znatno razlicite rezultate

U ovom koraku se sistematski parsiraju i transformišu najvažnije kolone iz skupa podataka o parfemima. Korišćenjem funkcije `literal_eval`, složene _string_ reprezentacije lista i rečnika (kao što su akordi, note i informacije o dizajnerima) konvertuju se u odgovarajuće _Python_ objekte, što omogućava efikasnu dalju obradu.

Za polja sa ocenama, izdvaja se i numerička vrednost ocene i broj glasova, čime se obezbeđuje da ulazni podaci modela odražavaju i kvalitet i popularnost svakog parfema. Sezonske i dnevne/noćne preferencije se parsiraju pomoću funkcije `parse_season_ratings()`.

Važno je napomenuti da akordi prvobitno nisu bili uključeni u ulazne karakteristike, ali su naknadno dodati radi usklađivanja sa prethodnim metodama. Uključivanje akorda nije značajno uticalo na performanse modela, što može ukazati na to da se glavni prediktivni potencijal nalazi u drugim atributima.

In [ ]:
data['Accords'] = data['Accords'].apply(literal_eval)
data['Notes'] = data['Notes'].apply(literal_eval)
data['Votes'] = data['Rating'].apply(lambda x: literal_eval(x)['votes'])
data['Rating'] = data['Rating'].apply(lambda x: literal_eval(x)['rating'])
data['Season ratings'] = data['Season ratings'].apply(parse_season_ratings)
data['Day ratings'] = data['Day ratings'].apply(parse_season_ratings)
data['Designers'] = data['Designers'].apply(literal_eval)

U ovom koraku se za svaki parfem iz skupa podataka kreira strukturirani zapis (`record`) koji obuhvata sve relevantne karakteristike: brend, akorde, pol, trajnost, _sillage_, ocenu, broj glasova, sezonske i dnevne/noćne preferencije.

Posebno se izdvajaju sve mirisne note parfema, koje se konsoliduju u jedinstvenu listu bez obzira na njihovu hijerarhiju (_top_, _middle_, _base_). Ova konsolidacija omogućava da se mirisni profil svakog parfema predstavi na jedinstven način, što olakšava dalju obradu, enkodiranje i primenu u modelu.

Rezultat je _DataFrame_ u kojem svaki red predstavlja jedan parfem sa svim ključnim atributima i kompletnom listom nota, spreman za naprednu analizu i treniranje neuronske mreže.

In [14]:
records = []
for _, row in data.iterrows():
    record = {
        "Brand": row["Brand"],
        "Accords": row["Accords"],
        "Gender": row["Gender"],
        "Longevity": row["Longevity"],
        "Sillage": row["Sillage"],
        "Rating": row["Rating"],
        "Votes": row["Votes"],
        "Season_Winter": row["Season ratings"].get("Winter", 0),
        "Season_Spring": row["Season ratings"].get("Spring", 0),
        "Season_Summer": row["Season ratings"].get("Summer", 0),
        "Season_Fall": row["Season ratings"].get("Fall", 0),
        "Day": row["Day ratings"].get("Day", 0),
        "Night": row["Day ratings"].get("Night", 0)
    }
    records.append(record)

structured_df = pd.DataFrame(records)
structured_df['All Notes'] = data['Notes'].apply(consolidate_notes) 

Mirisne note parfema su originalno predstavljene kao tekstualne vrednosti, što neuronska mreža ne može direktno da koristi. Da bi se omogućila obrada ovih podataka, svaka nota se mapira na jedinstveni numerički identifikator (_ID_) putem rečnika, čime se tekstualne informacije pretvaraju u numeričke nizove.

S obzirom da parfemi imaju različit broj nota, a neuronska mreža zahteva ulaze iste dužine, sve liste nota se skraćuju ili dopunjavaju nulama do fiksne dužine (20). Ova standardizacija omogućava da svaki parfem bude predstavljen nizom identifikatora iste veličine, što je optimalno za učenje modela. Eksperimentalno je utvrđeno da je dužina od 20 nota po parfemu dala najbolje rezultate, dok su kraći nizovi (npr. 10) dovodili do slabije performanse modela.

Ovim postupkom se obezbeđuje da podaci o notama budu u formatu pogodnom za embedding sloj neuronske mreže, čime se omogućava efikasno učenje reprezentacija mirisnih profila parfema.

In [ ]:
all_unique_notes = list(set(note for notes in structured_df['All Notes'] for note in notes)) 
note_to_id = {note: idx for idx, note in enumerate(all_unique_notes)} 
structured_df['Note_IDs'] = structured_df['All Notes'].apply(lambda notes: [note_to_id[note] for note in notes if note in note_to_id]) 
max_len = 20 
structured_df['Note_IDs_Padded'] = pad_sequences(structured_df['Note_IDs'], maxlen=max_len, padding='post').tolist() 

Akordi parfema su, kao i note, originalno predstavljeni kao tekstualne vrednosti, što nije pogodno za direktnu obradu u neuronskoj mreži. Svaki akord se mapira na jedinstveni numerički identifikator (_ID_) pomoću rečnika, čime se omogućava da se tekstualni podaci pretvore u numeričke nizove.

Pošto parfemi imaju različit broj akorda, sve liste akorda se skraćuju ili dopunjavaju nulama do fiksne dužine (10), kako bi svi ulazi u mrežu bili uniformni. Ova standardizacija omogućava da svaki parfem bude predstavljen nizom identifikatora iste veličine, što je optimalno za učenje modela. Eksperimentalno je utvrđeno da je dužina od 10 akorda po parfemu dala najbolje rezultate, dok su kraći nizovi (npr. 5) dovodili do slabije performanse modela.

Na ovaj način obezbeđeni su podaci o akordima tako da budu u formatu pogodnom za embedding sloj neuronske mreže, čime se omogućava efikasno učenje reprezentacija mirisnih karakteristika parfema.

In [16]:
all_unique_accords = list(set(accord for accords in structured_df['Accords'] for accord in accords)) 
accord_to_id = {accord: idx for idx, accord in enumerate(all_unique_accords)}
structured_df['Accord_IDs'] = structured_df['Accords'].apply(lambda accords: [accord_to_id[a] for a in accords if a in accord_to_id])
max_len_accords = 10  
structured_df['Accord_IDs_Padded'] = pad_sequences(structured_df['Accord_IDs'], maxlen=max_len_accords, padding='post').tolist()

Enkodiranje, odnsno pretvaranje kategoričnih vrednosti u numeričke:

In [17]:
label_encoder = LabelEncoder()
structured_df['Gender'] = label_encoder.fit_transform(structured_df['Gender'])
structured_df['Brand'] = label_encoder.fit_transform(structured_df['Brand'])

U ovom koraku iz strukturisanog skupa podataka izdvajaju se ulazne (`X`) i izlazne (`y`) promenljive za neuronsku mrežu. Kao cilj (`y`) koristi se ocena parfema, dok su ulazi (`X`) sve ostale relevantne karakteristike, izuzev kolona koje nisu pogodne za direktno treniranje, kao što su originalne liste nota i akorda.

Posebno se izdvajaju podaci o notama i akordima u obliku nizova fiksne dužine (`X_notes` i `X_accords`), što omogućava njihovu obradu u _embedding_ slojevima mreže. Numeričke kolone, poput trajnosti, _sillage_-a, broja glasova i sezonskih ocena, normalizuju se pomoću `StandardScaler`-a, čime se postiže prosečna vrednost nula i standardna devijacija jedan. Ova normalizacija olakšava učenje modela i dovodi do boljih rezultata u poređenju sa nenormalizovanim podacima.

Na kraju, ceo skup se deli na trening i test deo u odnosu 80:20, i to odvojeno za obične atribute (`X`) i za note/akorde (`X_notes`, `X_accords`).

Note i akordi se predstavljaju kao sekvence _ID_-jeva, gde svaki _ID_ označava jednu notu ili akord parfema. Pošto broj nota i akorda nije isti za svaki parfem, prethodno su skraćene ili dopunjene nulama do fiksne dužine, što je pogodno za _embedding_ sloj u mreži. Na taj način, `X_notes` i `X_accords` ulaze u _embedding_ slojeve, dok `X` ide u standardni _dense_ sloj, a kasnije se svi ulazi kombinuju i dalje obrađuju zajedno u mreži.

In [18]:
X = structured_df.drop(columns=['Rating', 'All Notes', 'Accords', 'Accord_IDs', 'Note_IDs', 'Note_IDs_Padded', 'Accord_IDs_Padded']) 

y = structured_df['Rating']
X_notes = np.array(structured_df['Note_IDs_Padded'].tolist())
X_accords = np.array(structured_df['Accord_IDs_Padded'].tolist())

numerical_features = ['Longevity','Sillage','Votes','Season_Winter','Season_Spring','Season_Summer','Season_Fall','Day','Night']
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_notes, X_test_notes = train_test_split(X_notes, test_size=0.2, random_state=42)
X_train_accords, X_test_accords = train_test_split(X_accords, test_size=0.2, random_state=42)

U ovom koraku se konstruiše neuronska mreža sa tri ulaza: prvi ulaz prima numeričke i enkodovane karakteristike parfema, drugi ulaz je niz _ID_-jeva nota, a treći niz _ID_-jeva akorda. Note i akordi se obrađuju kroz _embedding_ slojeve, gde se svaka nota mapira u vektor dimenzije 32, a svaki akord u vektor dimenzije 16. Dobijeni vektori se zatim spajaju (`flatten`) i kombinuju sa ostalim karakteristikama parfema.

Kombinovani ulaz prolazi kroz dva _dense_ sloja sa `ReLU` aktivacijom, što omogućava modelu da uči složene, nelinearne odnose između karakteristika i mirisnih profila. Na izlazu se dobija jedna vrednost koja predstavlja predviđenu ocenu parfema.

Model se kompajlira korišćenjem `Adam` optimizatora, koji je odabran zbog svoje efikasnosti i stabilnosti na heterogenim podacima. Za praćenje performansi tokom treniranja korišćene su metrike _MSE_ i _MAE_.

Eksperimentalno je utvrđeno da je _learning rate_ od 0.005 dao najbolje rezultate u poređenju sa drugim vrednostima (testiran je opseg od 0.001 do 0.01).

In [19]:
note_input = Input(shape=(max_len,))
note_embedding = Embedding(input_dim=len(all_unique_notes), output_dim=32)(note_input)
note_flattened = Flatten()(note_embedding)

accord_input = Input(shape=(max_len_accords,))
accord_embedding = Embedding(input_dim=len(all_unique_accords), output_dim=16)(accord_input)
accord_flattened = Flatten()(accord_embedding)

structured_input = Input(shape=(X_train.shape[1],))
concatenated = Concatenate()([structured_input, note_flattened, accord_flattened])

dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1)(dense_2)

ann_model = Model(inputs=[structured_input, note_input, accord_input], outputs=output)
ann_model.compile(optimizer=Adam(learning_rate=0.005), loss='mse', metrics=['mae'])

Model se trenira tokom 200 epoha sa veličinom _batch_-a 32.

Broj epoha je pažljivo odabran na osnovu eksperimentalnih rezultata: 200 epoha se pokazalo kao optimalan balans između tačnosti i brzine treniranja. Sa manjim brojem epoha (npr. 100) model nije postizao zadovoljavajuću tačnost, dok se sa većim brojem epoha (npr. 300) nije primećeno dodatno poboljšanje rezultata, već je treniranje postajalo sporije. Testiran je opseg vrednosti od 100 do 300 epoha, a 200 je izabrano kao najbolje rešenje za ovaj problem.

In [20]:
ann_model.fit([X_train, X_train_notes, X_train_accords], y_train, epochs=200, batch_size=32, verbose=0)

In [21]:
ann_loss, ann_mae = ann_model.evaluate([X_test, X_test_notes, X_test_accords], y_test, verbose=0)
ann_predictions = ann_model.predict([X_test, X_test_notes, X_test_accords]).flatten()
ann_rmse = np.sqrt(mean_squared_error(y_test, ann_predictions))

print(f"ANN Mean Absolute Error (MAE): {ann_mae}")
print(f"ANN Root Mean Squared Error (RMSE): {ann_rmse}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
ANN Mean Absolute Error (MAE): 0.21076291799545288
ANN Root Mean Squared Error (RMSE): 0.2703250910256294


U prethodnom primeru korišćene su konsolidovane note, gde su sve mirisne komponente parfema objedinjene u jednu listu bez obzira na hijerarhiju (_top_, _middle_, _base_). U narednom primeru note su tretirane odvojeno po slojevima, čime se modelu eksplicitno prenosi informacija o strukturi mirisnog profila.

Analizom rezultata pokazalo se da razlika u performansama modela između ova dva pristupa nije bila značajna. Konsolidacija nota pojednostavljuje ulazne podatke i olakšava obradu, dok odvojeno tretiranje nota može potencijalno omogućiti modelu da uoči nijanse između slojeva mirisnog profila.

Ovo ukazuje da, za zadatak predikcije ocene parfema, način predstavljanja nota (konsolidovano ili odvojeno) ne utiče drastično na tačnost modela.

In [6]:
records = []
for _, row in data.iterrows():
    record = {
        "Brand": row["Brand"],
        "Accords": row["Accords"],
        "Gender": row["Gender"],
        "Longevity": row["Longevity"],
        "Sillage": row["Sillage"],
        "Rating": row["Rating"],
        "Votes": row["Votes"],
        "Season_Winter": row["Season ratings"].get("Winter", 0),
        "Season_Spring": row["Season ratings"].get("Spring", 0),
        "Season_Summer": row["Season ratings"].get("Summer", 0),
        "Season_Fall": row["Season ratings"].get("Fall", 0),
        "Day": row["Day ratings"].get("Day", 0),
        "Night": row["Day ratings"].get("Night", 0)
    }
    records.append(record)

structured_df = pd.DataFrame(records)
structured_df['Top Notes'] = data['Notes'].apply(extract_top_notes) 
structured_df['Middle Notes'] = data['Notes'].apply(extract_middle_notes) 
structured_df['Base Notes'] = data['Notes'].apply(extract_base_notes) 

In [7]:
max_len = 10 

top_unique_notes = list(set(note for notes in structured_df['Top Notes'] for note in notes)) 
note_to_id = {note: idx for idx, note in enumerate(top_unique_notes)} 
structured_df['Top_Note_IDs'] = structured_df['Top Notes'].apply(lambda notes: [note_to_id[note] for note in notes if note in note_to_id]) 
structured_df['Top_Note_IDs_Padded'] = pad_sequences(structured_df['Top_Note_IDs'], maxlen=max_len, padding='post').tolist() 

middle_unique_notes = list(set(note for notes in structured_df['Middle Notes'] for note in notes)) 
note_to_id = {note: idx for idx, note in enumerate(middle_unique_notes)} 
structured_df['Middle_Note_IDs'] = structured_df['Middle Notes'].apply(lambda notes: [note_to_id[note] for note in notes if note in note_to_id])  
structured_df['Middle_Note_IDs_Padded'] = pad_sequences(structured_df['Middle_Note_IDs'], maxlen=max_len, padding='post').tolist() 

base_unique_notes = list(set(note for notes in structured_df['Base Notes'] for note in notes)) 
note_to_id = {note: idx for idx, note in enumerate(base_unique_notes)} 
structured_df['Base_Note_IDs'] = structured_df['Base Notes'].apply(lambda notes: [note_to_id[note] for note in notes if note in note_to_id]) 
structured_df['Base_Note_IDs_Padded'] = pad_sequences(structured_df['Base_Note_IDs'], maxlen=max_len, padding='post').tolist() 

In [8]:
all_unique_accords = list(set(accord for accords in structured_df['Accords'] for accord in accords)) 
accord_to_id = {accord: idx for idx, accord in enumerate(all_unique_accords)}
structured_df['Accord_IDs'] = structured_df['Accords'].apply(lambda accords: [accord_to_id[a] for a in accords if a in accord_to_id])
max_len_accords = 10  
structured_df['Accord_IDs_Padded'] = pad_sequences(structured_df['Accord_IDs'], maxlen=max_len_accords, padding='post').tolist()

In [9]:
label_encoder = LabelEncoder()
structured_df['Gender'] = label_encoder.fit_transform(structured_df['Gender'])
structured_df['Brand'] = label_encoder.fit_transform(structured_df['Brand'])

In [10]:
X = structured_df.drop(columns=['Rating', 'Top Notes', 'Middle Notes', 'Base Notes', 'Accords', 'Accord_IDs', 'Top_Note_IDs', 'Top_Note_IDs_Padded', 'Middle_Note_IDs', 'Middle_Note_IDs_Padded', 'Base_Note_IDs', 'Base_Note_IDs_Padded', 'Accord_IDs_Padded'])

y = structured_df['Rating']
X_top_notes = np.array(structured_df['Top_Note_IDs_Padded'].tolist())
X_middle_notes = np.array(structured_df['Middle_Note_IDs_Padded'].tolist())
X_base_notes = np.array(structured_df['Base_Note_IDs_Padded'].tolist())
X_accords = np.array(structured_df['Accord_IDs_Padded'].tolist())

numerical_features = ['Longevity','Sillage','Votes','Season_Winter','Season_Spring','Season_Summer','Season_Fall','Day','Night']
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_top_notes, X_test_top_notes = train_test_split(X_top_notes, test_size=0.2, random_state=42)
X_train_middle_notes, X_test_middle_notes = train_test_split(X_middle_notes, test_size=0.2, random_state=42)
X_train_base_notes, X_test_base_notes = train_test_split(X_base_notes, test_size=0.2, random_state=42)
X_train_accords, X_test_accords = train_test_split(X_accords, test_size=0.2, random_state=42)

In [11]:
top_note_input = Input(shape=(max_len,))
top_note_embedding = Embedding(input_dim=len(top_unique_notes), output_dim=32)(top_note_input)
top_note_flattened = Flatten()(top_note_embedding)

middle_note_input = Input(shape=(max_len,))
middle_note_embedding = Embedding(input_dim=len(middle_unique_notes), output_dim=32)(middle_note_input)
middle_note_flattened = Flatten()(middle_note_embedding)

base_note_input = Input(shape=(max_len,))
base_note_embedding = Embedding(input_dim=len(base_unique_notes), output_dim=32)(base_note_input)
base_note_flattened = Flatten()(base_note_embedding)

accord_input = Input(shape=(max_len_accords,))
accord_embedding = Embedding(input_dim=len(all_unique_accords), output_dim=16)(accord_input)
accord_flattened = Flatten()(accord_embedding)

structured_input = Input(shape=(X_train.shape[1],))
concatenated = Concatenate()([structured_input, top_note_flattened, middle_note_flattened, base_note_flattened, accord_flattened])

dense_1 = Dense(128, activation='relu')(concatenated)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1)(dense_2)

ann_model = Model(inputs=[structured_input, top_note_input, middle_note_input, base_note_input, accord_input], outputs=output)
ann_model.compile(optimizer=Adam(learning_rate=0.005), loss='mse', metrics=['mae'])

2025-09-14 21:42:14.160934: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:
ann_model.fit([X_train, X_train_top_notes, X_train_middle_notes, X_train_base_notes, X_train_accords], y_train, epochs=200, batch_size=32, verbose=0)

In [13]:
ann_loss, ann_mae = ann_model.evaluate([X_test, X_test_top_notes, X_test_middle_notes, X_test_base_notes, X_test_accords], y_test, verbose=0)
ann_predictions = ann_model.predict([X_test, X_test_top_notes, X_test_middle_notes, X_test_base_notes, X_test_accords]).flatten()
ann_rmse = np.sqrt(mean_squared_error(y_test, ann_predictions))
    
print(f"ANN Mean Absolute Error (MAE): {ann_mae}")
print(f"ANN Root Mean Squared Error (RMSE): {ann_rmse}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ANN Mean Absolute Error (MAE): 0.21068817377090454
ANN Root Mean Squared Error (RMSE): 0.27692273735441497
